# VMware EXSi Visualisations 1

In [1]:
import requests
import pandas as pd
from IPython.core.display import display, HTML
from modules.Network import *
display(HTML("<style>.container { width:70% !important; }</style>"))

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

We get the json file with the information from: https://wmwaredata.s3.us-east-2.amazonaws.com/machines.json

In [3]:
url = 'https://wmwaredata.s3.us-east-2.amazonaws.com/machines.json'
r = requests.get(url, allow_redirects=True)
open('data/machines.json', 'wb').write(r.content)

13379

In [4]:
json_file = 'data/machines.json' 
df = pd.read_json(json_file)
df

,BootTime,CommitedSpace,ConnectionState,CreateDate,Folder,Guest,GuestId,HardwareVersion,IPAddress,Id,...,MemoryMB,Name,Notes,NumCpu,NumVirtualDisks,PowerState,UNCommitedSpace,Uid,VMHost,Version
0,2021-01-07 13:36:24.248327+00:00,23708312044,connected,2021-01-07 13:36:24.265589+00:00,[datastore1] OPNsense/OPNsense.vmx,OPNsense,freebsd12_64Guest,NaN,None,564d29f0-0330-cbe9-ff62-322e2a8e89b0,...,2048,OPNsense,dont_delete,2,1,poweredOn,23708312044,521d330d-04c0-11d5-be0e-7818cfbb2407,'vim.HostSystem:ha-host',vmx-14
1,2021-02-08 12:53:59.688520+00:00,19411509752,connected,2021-02-08 12:53:59.709539+00:00,[datastore1] centos-testing/centos-testing.vmx,centos-testing,centos8_64Guest,NaN,91.109.25.76,564de62b-3ba3-d95e-e043-7c3c3835c472,...,2048,centos-testing,dont_delete,1,1,poweredOn,19411509752,522fb36e-4a08-5a8f-7812-1b0d1ceb535b,'vim.HostSystem:ha-host',vmx-14
2,2021-02-08 13:08:22.499520+00:00,2231640568,connected,2021-02-08 13:08:22.519463+00:00,[datastore1] TestInstallation/TestInstallation...,TestInstallation,centos8_64Guest,NaN,91.109.25.81,564d403b-69bb-ef7e-07a2-e064461cea9f,...,2048,TestInstallation,dont_delete,1,1,poweredOn,2231640568,52de03ee-c829-7e4f-36e8-f5ec021d408a,'vim.HostSystem:ha-host',vmx-14
3,None,52428811647,connected,2021-01-21 17:50:38.270450+00:00,[datastore1] packer-proxy-rebuild/packer-proxy...,packer-proxy-rebuild,ubuntu64Guest,NaN,None,,...,8192,packer-proxy-rebuild,,2,1,poweredOff,52428811647,,'vim.HostSystem:ha-host',vmx-13
4,2021-02-04 13:36:34.225529+00:00,36593834190,connected,2021-02-04 13:33:31.307230+00:00,[datastore1] Controller/Controller.vmx,Controller,ubuntu64Guest,NaN,91.109.26.21,564d22d6-99de-8186-43ac-b01189fd0193,...,4096,Controller,,2,1,poweredOn,36593834190,527c8e8f-22a2-aa8e-f14c-943aae2a9769,'vim.HostSystem:ha-host',vmx-14
5,2021-02-04 12:54:16.039620+00:00,11834237420,connected,2021-02-04 13:43:03.387354+00:00,[datastore1] file-drop-1/file-drop-1.vmx,file-drop-1,ubuntu64Guest,NaN,10.42.0.1,564db201-59fd-7e3a-90be-bf81ddd5e8cd,...,2048,file-drop-1,,2,1,poweredOn,11834237420,523fa171-d3e6-090c-c37d-b9fdc28aaa7a,'vim.HostSystem:ha-host',vmx-13
6,None,6595756315,connected,2021-02-08 12:29:12.227368+00:00,[datastore1] ICAP2/ICAP2.vmx,ICAP2,ubuntu64Guest,NaN,None,564d1e80-e0fe-e6f5-dbfc-43d5ad229062,...,8192,ICAP2,,4,1,poweredOff,6595756315,52b45454-872d-cf3e-5b37-ec3792e4fc89,'vim.HostSystem:ha-host',vmx-07
7,None,6902784492,connected,2021-02-04 21:39:44.119473+00:00,[datastore1] IQT_ICAP_Test_020421/IQT_ICAP_Tes...,IQT_ICAP_Test_020421,ubuntu64Guest,NaN,None,564dd11c-291b-8450-abaa-d1fea268f34d,...,2048,IQT_ICAP_Test_020421,,1,1,poweredOff,6902784492,526bd8d8-e8e0-2f40-c226-d328007a14ac,'vim.HostSystem:ha-host',vmx-07
8,2021-02-07 10:38:40.660762+00:00,69698858335,connected,2021-02-07 10:38:40.682321+00:00,[datastore1] logtrixia/logtrixia.vmx,logtrixia,ubuntu64Guest,NaN,91.109.26.22,564d4d69-9038-90dd-030e-102a7251d021,...,16384,logtrixia,"Faisal, Monitoring Server, Demo",4,1,poweredOn,69698858335,,'vim.HostSystem:ha-host',vmx-13
9,2021-02-07 16:17:51.478830+00:00,18401468908,connected,2021-02-07 16:17:51.495868+00:00,[datastore1] test-srevice-ova/test-srevice-ova...,test-srevice-ova,ubuntu64Guest,NaN,172.18.0.1,564d22df-f8c8-6047-363c-ae9877bcde13,...,8192,test-srevice-ova,,2,1,poweredOn,18401468908,52510480-c357-6374-272e-50802f0bd98a,'vim.HostSystem:ha-host',vmx-07


## Workflow

In [5]:
%%html
<div id="mynetwork"></div>

In [6]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

var SHAPE = "circle", //"box", //"square"
    SHAPE2 = "box",
    BLUE = "#99CDFF",
    YELLOW = "#FEEBA7",
    GREEN = "#C6E7B0",
    PURPLE = "#DAADF0",
    ORANGE = "#FFC266",
    PINK = "#FFBBB2",
    BLACK = "#2B1B17",
    BROWN = "#ADAD85",
    RED = "#FF6666";
    
require(['vis'], function(vis){    
    
    // create an array with nodes
    var nodes = new vis.DataSet([ 
      { id: 0, label: "Name\nId", group: "Name" },  
      { id: 1, label: "GuestId", group: "GuestId" },  
      { id: 2, label: "NumCpu", group: "NumCpu" }, 
      { id: 3, label: "MemoryMB",  group: "MemoryMB" }, 
      { id: 4, label: "PowerState",  group: "PowerState" },
      { id: 5, label: "HardwareVersion", group: "HardwareVersion" },
      { id: 6, label: "Version", group: "Version" }, 
      { id: 7, label: "ConnectionState", group: "ConnectionState" }, 
      //{ id: 8, label: "VMResourceConfiguration", group: "VMResourceConfiguration" },              
    ]);

    // create an array with edges
    var edges = new vis.DataSet([
      { id: 1, from: 0, to: 1, arrows: "to", color: BLACK },
      { id: 2, from: 0, to: 2, arrows: "to", color: BLACK },
      { id: 3, from: 0, to: 3, arrows: "to", color: BLACK },
      { id: 4, from: 0, to: 4, arrows: "to", color: BLACK },
      { id: 5, from: 0, to: 5, arrows: "to", color: BLACK },
      { id: 6, from: 0, to: 6, arrows: "to", color: BLACK },
      { id: 7, from: 0, to: 7, arrows: "to", color: BLACK },
    ]);    

    // create a network
    var data = {
      nodes: nodes,
      edges: edges,
    };
    var container = document.getElementById("mynetwork");
    var options = {      
      width: '1200px',
      height: '400px',
      physics: true,
      groups: {
        Name: {
          shape: SHAPE2,
          color: BLUE,
        },       
        GuestId: {
          shape: SHAPE2,
          color: YELLOW,
        },
        NumCpu: {
          shape: SHAPE,
          color: PINK,
        },
        MemoryMB: {
          shape: SHAPE,
          color: GREEN,
        },
        PowerState: {
          shape: SHAPE2,
          color: PURPLE,
        },
        HardwareVersion: {
          shape: SHAPE2,
          color: RED,
        },
        Version: {
          shape: SHAPE,
          color: ORANGE,
        },
        ConnectionState: {
          shape: SHAPE2,
          color: BROWN,
        },
      },    
    }; 
    var network = new vis.Network(container, data, options);

});

<IPython.core.display.Javascript object>

## Option 1: With Hierarchy

In [7]:
# CREATE JSON FILE WITH NODES AND EDGES FOR THE HIERARCHY NETWORK

network1 = Network()

network1.groups = ['Name', 'GuestId', 'NumCpu', 'MemoryMB', 'PowerState', 'HardwareVersion', 'Version', 'ConnectionState']


network1.groupSettings = group_settings_VM

for i in range(len(df)):
    
    # NODES   
    labels = []
    
    label = df.iloc[i]['Name'] + "\n" + df.iloc[i]['Id']
    label = wrap_by_char(label, n=15)
    labels.append(label)
    network1.add_node(label, group='Name')
    
    for group in network1.groups[1:]:
        label = group + ':\n' + str(df.iloc[i][group])
        labels.append(label)
        network1.add_node(label, group=group)
        
    #label = 'VMResourceConfiguration:\n' + wrap_by_word(df.iloc[i]['VMResourceConfiguration'], n=1)
    #labels.append(label)
    #network1.add_node(label, group="VMResourceConfiguration")   
    
    # EDGES
    for label in labels[1:]:
        network1.add_edge(network1.nodesIds[labels[0]][0], network1.nodesIds[label][0])

network1.save_to_json('data/nodes_edges_1.json')

In [8]:
%%html
<div id="mynetwork2"></div>

In [9]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});


require(['vis'], function(vis){
    
    var json = $.getJSON("data/nodes_edges_1.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
    
    var options = {
      width: '1200px',
      height: '900px',
      locale: 'en',
      physics: false,
      interaction: {
        hover:true, 
        tooltipDelay: 300
      },        
      layout: {
        randomSeed: 1,
        improvedLayout: true,
        hierarchical: {
          enabled: true,
          direction: 'UD',
          nodeSpacing: 200,
          levelSeparation: 250,  
          sortMethod: 'directed',
        },
      },
    }; 
    
    var container = document.getElementById("mynetwork2");
    
});

<IPython.core.display.Javascript object>

## Option 2: No Hierarchy

In [10]:
# CREATE JSON FILE WITH NODES AND EDGES FOR THE NO HIERARCHY NETWORK

network2 = Network()

network2.groups = ['Name', 'GuestId', 'NumCpu', 'MemoryMB', 'PowerState', 'HardwareVersion', 'Version', 'ConnectionState']

network2.groupSettings = group_settings_VM


BLUE = "#99CDFF"
YELLOW = "#FEEBA7"
PINK = "#FFBBB2"
PURPLE = "#DAADF0"
ORANGE = "#FFC266"
GREEN = "#C6E7B0"
BROWN = "#ADAD85"
RED = "#FF6666"
AQUAMARINE = "#5CD6D6"

SHAPE = "circle"
SHAPE2 = "box"
SHAPE3 = "default"


labels = ["Name\nId", "GuestId", "NumCpu", "MemoryMB", "PowerState", "HardwareVersion", "Version", "ConnectionState"]
colors = [BLUE, YELLOW, PINK, GREEN, PURPLE, RED, ORANGE, BROWN]
shapes = [SHAPE2, SHAPE2, SHAPE3, SHAPE3, SHAPE2, SHAPE2, SHAPE, SHAPE2]

x = -950
y = -950
step = 70    


# REFERENCE NODES
for label, i, color, shape in zip(labels, range(8), colors, shapes):
    network2.add_node(label, group='Reference', x=x, y=y+i*step, color=color, shape=shape)
    
    
for i in range(len(df)):
    
    # NODES   
    labels = []
    
    label = df.iloc[i]['Name'] + "\n" + df.iloc[i]['Id']
    label = wrap_by_char(label, n=15)
    labels.append(label)
    network2.add_node(label, group='Name')
    
    for group in network2.groups[1:]:
        label = group + ':\n' + str(df.iloc[i][group])
        labels.append(label)
        network2.add_node(label, group=group)
        
    #label = 'VMResourceConfiguration:\n' + wrap_by_word(df.iloc[i]['VMResourceConfiguration'], n=1)
    #labels.append(label)
    #network2.add_node(label, group="VMResourceConfiguration")   
    
    # EDGES
    for label in labels[1:]:
        network2.add_edge(network2.nodesIds[labels[0]][0], network2.nodesIds[label][0])

network2.save_to_json('data/nodes_edges_2.json')

In [11]:
%%html
<div id="mynetwork3"></div>

In [12]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});


require(['vis'], function(vis){ 
    
    var json = $.getJSON("data/nodes_edges_2.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
       
    var options = {      
      width: '1200px',
      height: '1000px',
      physics: true,
    };    
    
    var container = document.getElementById("mynetwork3");
});

<IPython.core.display.Javascript object>